<a href="https://colab.research.google.com/github/nrohrbach/sharedmobility_monitoring/blob/main/sharedmobility_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sharedmobility.ch - Validation of GBFS-Feeds

In [1]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 7.8 MB/s eta 0:00:00


In [17]:
import requests
import pandas as pd
from datetime import datetime
import json

In [130]:
# GBFS Feeds abfragen
url = 'https://gbfs.prod.sharedmobility.ch/v2/gbfs'
feeds = requests.get(url).json()

In [131]:
# Alle GBFS Feeds als Dataframe laden
id = [s['id'] for s in feeds['systems']]
url = [s['url'] for s in feeds['systems']]
gbfsfeeds = pd.DataFrame(list(zip(id, url)),columns =['provider', 'gbfsurl'])
providers = list(gbfsfeeds['provider'])

In [98]:
# Nur einen Provider temporär einstellen!
providers = ['bolt_zurich','velospot','carvelo2go','bird-biel']
providers

['bolt_zurich', 'velospot', 'carvelo2go', 'bird-biel']

In [132]:
# Validieren!
JsonResponse = []
providerList2 = []
HasErrors = []
NumberOfErrors = []

for provider in providers:
    try:
      headers = {
          'accept': '*/*',
          'Content-Type': 'application/json',
          'User-Agent': 'geoinformation@bfe.admin.ch',
      }

      json_data = {
          'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/'+ provider +'/gbfs',
          'options': {
              'freefloating': False,
              'docked': False,
              'version': None,
              'auth': {
                  'type': None,
                  'basicAuth': {
                      'user': None,
                      'password': None,
                  },
                  'bearerToken': {
                      'token': None,
                  },
                  'oauthClientCredentialsGrant': {
                      'user': None,
                      'password': None,
                      'tokenUrl': None,
                  },
              },
          },
      }
      response = requests.post('https://gbfs-validator.netlify.app/.netlify/functions/validator', headers=headers, json=json_data).json()

      providerList2.append(provider)
      JsonResponse.append(response)
      HasErrors.append(response['summary']['hasErrors'])
      NumberOfErrors.append(response['summary']['errorsCount'])

    except:
      providerList2.append(provider)
      JsonResponse.append(response)
      HasErrors.append('nan')
      NumberOfErrors.append('nan')

In [167]:
ValidationResults = pd.json_normalize(JsonResponse)

In [168]:
ValidationResults['provider'] = providerList2

In [169]:
ValidationResults['Validator'] = 'https://gbfs-validator.netlify.app/validator?url=https://gbfs.prod.sharedmobility.ch/v2/gbfs/'+ ValidationResults['provider']+'/gbfs'

In [170]:
ValidationResults = ValidationResults.drop(columns=['files','summary.validatorVersion','summary.version.detected','errorMessage'])

In [ ]:
ValidationResults

In [171]:
ValidationResults.to_csv("GBFS_validated.csv",header=True,index=False)